In [1]:
import matplotlib.pyplot as plt
import numpy as np

def phi(t):
    return np.where((t >= 0) & (t < 1), 1.0, 0.0)

def psi(n, k, t):
    # Haar wavelet on dyadic interval [k/2^n, (k+1)/2^n)
    left = k / 2**n
    mid = (k + 0.5) / 2**n
    right = (k + 1) / 2**n
    out = np.zeros_like(t)
    amp = 2**(n/2)
    out[(t >= left) & (t < mid)] = amp
    out[(t >= mid) & (t < right)] = -amp
    return out

# Plot example: indicator [0,1/2) and decomposition with phi + psi_{0,0}
t = np.linspace(0, 1, 500)

indicator = np.where((t >= 0) & (t < 0.5), 1.0, 0.0)
approx = 0.5*phi(t) + (1/(2*np.sqrt(2)))*psi(0,0,t)

plt.figure(figsize=(10,6))
plt.plot(t, indicator, label=r'Indicator $1_{[0,1/2)}$', lw=3)
plt.plot(t, approx, '--', label=r'Expansion $\tfrac{1}{2}\phi + \tfrac{1}{2\sqrt{2}}\psi_{0,0}$', lw=2)
plt.plot(t, phi(t), ':', label=r'$\phi$ (scaling)', lw=1.5)
plt.plot(t, psi(0,0,t), ':', label=r'$\psi_{0,0}$ (wavelet)', lw=1.5)

plt.ylim(-1.5, 1.5)
plt.legend(fontsize=12)
plt.title("Haar Expansion of Dyadic Indicator Function", fontsize=14)
plt.xlabel("t")
plt.ylabel("Value")
plt.grid(True, alpha=0.3)
plt.show()


ValueError: 
\tfrac{1}{2}\phi + \tfrac{1}{2\sqrt{2}}\psi_{0,0}
^
ParseFatalException: Unknown symbol: \tfrac, found '\'  (at char 0), (line:1, col:1)

<Figure size 1000x600 with 1 Axes>

In [29]:
import os
import pickle
import wandb
import jax
from datetime import datetime
from helper import parse_cfg
from omegaconf import OmegaConf
from flax.training import checkpoints
from learning.agents.ppo import networks as ppo_networks
from learning.agents.sac import networks as sac_networks
from learning.agents.td3 import networks as td3_networks
import mediapy as media
import hydra
import functools
from learning.configs import dm_control_training_config, locomotion_training_config
from brax.training.acme import running_statistics
from mujoco_playground._src.wrapper import BraxDomainRandomizationVmapWrapper
from etils import epath
from omegaconf import OmegaConf
from mujoco_playground._src import mjx_env
from typing import Any, Callable, Dict, Optional, Type, Union, Tuple
from mujoco import mjx
import numpy
from custom_envs import registry, dm_control_suite, locomotion
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ['XLA_FLAGS'] = xla_flags
os.environ["MUJOCO_GL"] = "egl"
os.environ['JAX_PLATFORM_NAME'] = 'gpu'
os.environ['CUDA_VISIBLE_DEVICES'] = '6'  # change to your GPU id


In [30]:
from mujoco_playground._src.wrapper import Wrapper
class BraxDomainRandomizationWrapper(Wrapper):
  """Brax wrapper for domain randomization."""
  def __init__(
      self,
      env: mjx_env.MjxEnv,
      randomization_fn: Callable[[mjx.Model], Tuple[mjx.Model, mjx.Model]],
  ):
    super().__init__(env)
    self._mjx_model, self._in_axes = randomization_fn(self.env.mjx_model)
    self.env.unwrapped._mjx_model = self._mjx_model

  # def _env_fn(self, mjx_model: mjx.Model) -> mjx_env.MjxEnv:
  #   env = self.env
  #   env.unwrapped._mjx_model = mjx_model
  #   return env

  def reset(self, rng: jax.Array) -> mjx_env.State:
    # def reset(mjx_model, rng):
    #   env = self._env_fn(mjx_model=mjx_model)
    #   return env.reset(rng)

    state = self.env.reset(rng)
    return state

  def step(self, state: mjx_env.State, action: jax.Array) -> mjx_env.State:
    # def step(mjx_model, s, a):
    #   env = self._env_fn(mjx_model=mjx_model)
    #   return env.step(s, a)

    res = self.env.step(state, action)
    return res


In [31]:
CAMERAS = {
    "AcrobotSwingup": "fixed",
    "AcrobotSwingupSparse": "fixed",
    "BallInCup": "cam0",
    "CartpoleBalance": "fixed",
    "CartpoleBalanceSparse": "fixed",
    "CartpoleSwingup": "fixed",
    "CartpoleSwingupSparse": "fixed",
    "CheetahRun": "side",
    "FingerSpin": "cam0",
    "FingerTurnEasy": "cam0",
    "FingerTurnHard": "cam0",
    "FishSwim": "fixed_top",
    "HopperHop": "cam0",
    "HopperStand": "cam0",
    "HumanoidStand": "side",
    "HumanoidWalk": "side",
    "HumanoidRun": "side",
    "PendulumSwingup": "fixed",
    "PointMass": "cam0",
    "ReacherEasy": "fixed",
    "ReacherHard": "fixed",
    "SwimmerSwimmer6": "tracking1",
    "WalkerRun": "side",
    "WalkerWalk": "side",
    "WalkerStand": "side",    
    "Go1Handstand": "side",
    "Go1JoystickRoughTerrain": "track",
}


In [32]:

from agents.flowsac import networks as flowsac_networks
from agents.rambo import networks as rambo_networks

x_data, y_data, y_dataerr = [], [], []
times = [datetime.now()]

def evaluate(cfg):

    # Load environment
    env = registry.load(cfg.task)
    env_cfg = registry.get_default_config(cfg.task)
    print("env nbody", env._mj_model.nbody)
    print("env nv", env._mj_model.nv)
    obs_size = env.observation_size
    act_size = env.action_size
    rng = jax.random.PRNGKey(cfg.eval_seed)
    
    if cfg.randomization:
        path = epath.Path(".").resolve()

        randomization_fn = registry.get_domain_randomizer_eval(cfg.task)
        rng, dynamics_rng = jax.random.split(rng)
        randomization_fn = functools.partial(randomization_fn, rng = dynamics_rng, params=env.dr_range)
        env = BraxDomainRandomizationWrapper(
            env,
            randomization_fn=randomization_fn,
        )
    if cfg.policy == "sac":
        if cfg.task in dm_control_suite._envs:
            sac_params = dm_control_training_config.brax_sac_config(cfg.task)
        elif cfg.task in locomotion._envs:
            sac_params = locomotion_training_config.locomotion_sac_config(cfg.task)
        sac_training_params = dict(sac_params)
        network_factory = sac_networks.make_sac_networks
        if "network_factory" in sac_params:
            del sac_training_params["network_factory"]
            network_factory = functools.partial(
                sac_networks.make_sac_networks,
                **sac_params.network_factory
            )
        sac_network = network_factory(
            observation_size=obs_size,
            action_size=act_size,
            preprocess_observations_fn=running_statistics.normalize if sac_params.normalize_observations else None, 
        )
        make_policy_fn = sac_networks.make_inference_fn(sac_network)
    elif cfg.policy == "ppo":
        if cfg.task in dm_control_suite._envs:
            ppo_params = dm_control_training_config.brax_ppo_config(cfg.task)
        elif cfg.task in locomotion._envs:
            ppo_params = locomotion_training_config.locomotion_ppo_config(cfg.task)
        network_factory = ppo_networks.make_ppo_networks
        if "network_factory" in ppo_params:
            network_factory = functools.partial(
                ppo_networks.make_ppo_networks,
                **ppo_params.network_factory
            )
        ppo_network = network_factory(
            observation_size=obs_size,
            action_size=act_size,
            preprocess_observations_fn=running_statistics.normalize if ppo_params.normalize_observations else None,
        )

        make_policy_fn = ppo_networks.make_inference_fn(ppo_network)
    elif cfg.policy == "rambo":
        rambo_params = brax_rambo_config(cfg.task)
        rambo_training_params = dict(rambo_params)
        network_factory = rambo_networks.make_rambo_networks
        if "network_factory" in rambo_params:
            del rambo_training_params["network_factory"]
            network_factory = functools.partial(
                rambo_networks.make_rambo_networks,
                **rambo_params.network_factory
            )
        rambo_network = network_factory(
            observation_size=obs_size,
            action_size=act_size,
            preprocess_observations_fn=running_statistics.normalize if rambo_params.normalize_observations else None,
        )
        make_policy_fn = rambo_networks.make_inference_fn(rambo_network)
    elif cfg.policy == "flowsac":
        flowsac_params = brax_flowsac_config(cfg.task)
        flowsac_training_params = dict(flowsac_params)
        network_factory = flowsac_networks.make_flowsac_networks
        if "network_factory" in flowsac_params:
            del flowsac_training_params["network_factory"]
            network_factory = functools.partial(
                flowsac_networks.make_flowsac_networks,
                **flowsac_params.network_factory
            )
        flowsac_network = network_factory(
            observation_size=obs_size,
            action_size=act_size,
            preprocess_observations_fn=running_statistics.normalize if flowsac_params.normalize_observations else None,
            dynamics_param_size=len(env.dr_range)
        )
        make_policy_fn = flowsac_networks.make_inference_fn(flowsac_network)
    elif cfg.policy == "td3":
        if cfg.task in dm_control_suite._envs:
            td3_params = dm_control_training_config.brax_td3_config(cfg.task)
        elif cfg.task in locomotion._envs:
            td3_params= locomotion_training_config.locomotion_td3_config(cfg.task)
        td3_training_params = dict(td3_params)
        network_factory = td3_networks.make_td3_networks
        if "network_factory" in td3_params:
            del td3_training_params["network_factory"]
            network_factory = functools.partial(
                td3_networks.make_td3_networks,
                **td3_params.network_factory
            )
        td3_network = network_factory(
            observation_size=obs_size,
            action_size=act_size,
            preprocess_observations_fn=running_statistics.normalize if td3_params.normalize_observations else None, 
        )
        make_policy_fn = td3_networks.make_inference_fn(td3_network)
    # Load saved parameters
    save_dir = os.path.join(cfg.work_dir, "models")
    print(f"Loading parameters from {save_dir}")
    with open(os.path.join(save_dir, f"{cfg.policy}_params_latest.pkl"), "rb") as f:
        params = pickle.load(f)

    jit_inference_fn = jax.jit(make_policy_fn(params,deterministic=True))

    jit_reset = jax.jit(env.reset)
    jit_step = jax.jit(env.step)

    rollout = []
    rewards = []
    rng =jax.random.PRNGKey(cfg.eval_seed)
    reset_rng, rng = jax.random.split(rng)
    reset_rngs = jax.random.split(reset_rng, 10)
    for i in range(10):
        # Evaluation loop
        
        state = jit_reset(reset_rngs[i])
        total_reward = 0.0
        
        rollout = [state]

        for _ in range(env_cfg.episode_length):
            act_rng, rng = jax.random.split(rng)
            action, info = jit_inference_fn(state.obs, act_rng)
            state = jit_step(state, action)
            rollout.append(state)
            total_reward += state.reward
        rewards.append(total_reward)
    frames = env.render(rollout, camera=CAMERAS[cfg.task])
    media.show_video(frames, fps=1.0 / env.dt)
    import numpy as np
    print(f"Total reward: {np.array(rewards).mean()}")


In [ ]:

cfg_path= epath.Path(".").resolve()
print(cfg_path)
cfg_path = os.path.join(cfg_path, "config.yaml")
# cfg = compose(config_name="config.yaml")
cfg = OmegaConf.load(cfg_path)
cfg.task="Go1JoystickRoughTerrain"
cfg.policy="sac"
cfg.seed=0
cfg.randomization=False

print("cfg:", cfg)
cfg = parse_cfg(cfg)
cfg.eval_seed= 55
print("work dir", cfg.work_dir)

# cfg = OmegaConf.load(cfg_path)

evaluate(cfg)
#843.25


/raid/users/tjrcjf410/distributionally_robust_learning/learning
cfg: {'benchmark': 'dm_control', 'task': 'Go1JoystickRoughTerrain', 'obs': 'state', 'exp_name': 'test', 'checkpoint': '???', 'eval_episodes': 1, 'eval_pi': True, 'eval_value': True, 'eval_freq': 50000, 'eval_with_training_env': False, 'policy': 'sac', 'asymmetric_critic': False, 'randomization': False, 'eval_randomization': True, 'wandb_project': 'wdsac-exp', 'wandb_entity': 'tjrcjf410-seoul-national-university', 'wandb_silent': False, 'use_wandb': True, 'save_csv': True, 'save_video': True, 'save_agent': True, 'seed': 0, 'work_dir': '???', 'task_title': '???', 'multitask': '???', 'tasks': '???', 'obs_shape': '???', 'action_dim': '???', 'episode_length': '???', 'obs_shapes': '???', 'action_dims': '???', 'episode_lengths': '???', 'seed_steps': '???', 'bin_size': '???', 'real_ratio': '???', 'rollout_length': '???', 'adv_weight': '???', 'batch_size': '???', 'rollout_batch_size': '???', 'n_nominals': '???', 'delta': '???', 'la

100%|██████████| 1001/1001 [00:02<00:00, 365.36it/s]


This browser does not support the video tag.


Total reward: 12.282464981079102


: 

In [ ]:
env._config.command_config


a:
- 1.5
- 0.8
- 1.2
b:
- 0.9
- 0.25
- 0.5

In [34]:
randomization_fn = registry.get_domain_randomizer_eval(cfg.task)
rng = jax.random.PRNGKey(cfg.seed)
rng, dynamics_rng = jax.random.split(rng)
env = registry.load(cfg.task)
randomization_fn = functools.partial(randomization_fn, rng = dynamics_rng, params=env.dr_range)
env = BraxDomainRandomizationWrapper(
                env,
                randomization_fn=randomization_fn,
            )


AttributeError: 'Joystick' object has no attribute 'dr_range'

In [ ]:
env.dr_range


(Array([ 0.9,  0. ,  0. , -0.3, -0.3,  0.5,  0.5], dtype=float32),
 Array([1. , 1. , 1. , 0.3, 0.3, 1.5, 1.5], dtype=float32))